In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
# define image size
img_rows=28
img_cols=28
# load dataset
(X_train , y_train) ,(X_test , y_test ) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

# reshape
X_train= X_train.reshape(X_train.shape [0], img_rows , img_cols , 1)
X_test= X_test.reshape(X_test.shape [0], img_rows , img_cols , 1)
# one hot encoded
y_train= np_utils.to_categorical(y_train )
y_test= np_utils.to_categorical(y_test)
# fix random seed for reproducibility
seed = 100
np.random.seed(seed)
num_classes= 10

# create CNN model
def cnn_model():
    # define model9
    model = Sequential()
    model.add(Convolution2D(32, kernel_size =(5, 5),
                            #border_mode=='valid', 
                            strides=(1, 1), 
                            input_shape=(img_rows , img_cols , 1), 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size =(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(127, activation='relu'))
    model.add(Dense(num_classes , activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# build the model
model = cnn_model()
# Fit the model
disp= model.fit(X_train , y_train ,validation_data=(X_test , y_test ),nb_epoch=10,batch_size=200,verbose=1)
# Final evaluation of the model
scores =model.evaluate(X_test , y_test , verbose=0)
print("loss: %.2f" % scores[0])
print("acc: %.2f" % scores[1])

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
# define image size
img_rows=28
img_cols=28
# load dataset
(X_train , y_train) ,(X_test , y_test ) = mnist.load_data()
'''print(X_train.shape)
print(y_train.shape)
print(X_train)'''

In [ ]:
img_1 = 32
img_2 = 32
img_3 = 3

X_train, X_test = X_train / 255.0, X_test / 255.0
# reshape
X_train= X_train.reshape(X_train.shape [0], img_1 , img_2 , img_3)
X_test= X_test.reshape(X_test.shape [0], img_1 , img_2 , img_3)
# one hot encoded
y_train= np_utils.to_categorical(y_train )
y_test= np_utils.to_categorical(y_test)
# fix random seed for reproducibility
seed = 100
np.random.seed(seed)
num_classes= 10

# create CNN model
def cnn_model():
    # define model9
    model = Sequential()
    model.add(Convolution2D(32, kernel_size =(5, 5),#아마 여기가 문제
                            #border_mode=='valid', 
                            strides=(1, 1), 
                            input_shape=(img_1 , img_2 , img_3), 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size =(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(127, activation='relu'))
    model.add(Dense(num_classes , activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# build the model
model = cnn_model()
# Fit the model
disp= model.fit(X_train , y_train ,validation_data=(X_test , y_test),nb_epoch=10,batch_size=200,verbose=1)
# Final evaluation of the model
scores =model.evaluate(X_test , y_test , verbose=0)
print("loss: %.2f" % scores[0])
print("acc: %.2f" % scores[1])

In [ ]:
import sys, os
import numpy as np
from keras.datasets import cifar10
import matplotlib.pyplot as plt

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """
    다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2 * pad - filter_h) // stride + 1
    out_w = (W + 2 * pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self. stride)
        out_w = int(1 + (H + 2*self.pad - FW) / self. stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = self.stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (H - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최댓값 (2)
        out = np.max(col, axis=1)  # axis=0 : 열, axis=1 : 행
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        
        input_size = input_dim[1]
        
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                          conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads   

# load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Scale the data
X_train = X_train / 255
X_test = X_test / 255

y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test, 10)

#Create Model
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

#Augment Data
batch_size = 32
data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(X_train, y_cat_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

r = model.fit(train_generator, 
              epochs=50,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_test, y_cat_test)
             )

evaluation = model.evaluate(X_test, y_cat_test)
print(f'Test Accuracy : {evaluation[1] * 100:.2f}%')

Epoch 1/50
1563/1563 [==============================] - 735s 470ms/step - loss: 1.6367 - acc: 0.4080 - precision: 0.6190 - recall: 0.1913 - val_loss: 1.3894 - val_acc: 0.5200 - val_precision: 0.6578 - val_recall: 0.3872
Epoch 2/50
1563/1563 [==============================] - 789s 505ms/step - loss: 1.2247 - acc: 0.5627 - precision: 0.7246 - recall: 0.4002 - val_loss: 1.0723 - val_acc: 0.6361 - val_precision: 0.7413 - val_recall: 0.5468
Epoch 3/50
1563/1563 [==============================] - 733s 469ms/step - loss: 1.0344 - acc: 0.6406 - precision: 0.7740 - recall: 0.5104 - val_loss: 0.9811 - val_acc: 0.6665 - val_precision: 0.7916 - val_recall: 0.5716
Epoch 4/50
1563/1563 [==============================] - 659s 422ms/step - loss: 0.9246 - acc: 0.6864 - precision: 0.7996 - recall: 0.5753 - val_loss: 0.7797 - val_acc: 0.7353 - val_precision: 0.8262 - val_recall: 0.6502
Epoch 5/50
1563/1563 [==============================] - 692s 443ms/step - loss: 0.8448 - acc: 0.7117 - precision: 0.8137

KeyboardInterrupt: 

In [4]:
evaluation = model.evaluate(X_test, y_cat_test)
print(f'Test Accuracy : {evaluation[1] * 100:.2f}%')

10000/10000 [==============================] - 18s 2ms/sample - loss: 0.4804 - acc: 0.8391 - precision: 0.8866 - recall: 0.7975
Test Accuracy : 83.91%
